In [175]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [176]:
best_parameters = gscv.best_params_
# best_parameters

In [177]:
traindata = pd.read_csv('Train.csv')
riders = pd.read_csv('Riders.csv')

In [178]:
traindata.shape

(21201, 29)

In [179]:
testdata = traindata[5000:].reset_index(inplace = False)
testdata.drop(columns='index', inplace=True)

In [180]:
test

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_5073,User_Id_2200,Bike,3,Business,25,4,4:18:56 PM,25,4,...,6:22:17 PM,8,NaN,NaN,-1.301446,36.766138,-1.290894,36.822971,Rider_Id_436,1861
1,Order_No_16569,User_Id_972,Bike,3,Business,3,1,12:28:16 PM,3,1,...,2:34:37 PM,15,20.7,NaN,-1.286203,36.815297,-1.308100,36.699964,Rider_Id_272,5216
2,Order_No_17965,User_Id_2587,Bike,3,Business,5,5,4:50:48 PM,5,5,...,5:59:46 PM,22,26.2,NaN,-1.288780,36.816831,-1.316554,36.700067,Rider_Id_456,1997
3,Order_No_20647,User_Id_2483,Bike,1,Business,16,1,12:33:26 PM,16,1,...,1:53:55 PM,12,19.4,NaN,-1.255956,36.789867,-1.284269,36.739776,Rider_Id_536,3949
4,Order_No_23907,User_Id_1502,Bike,3,Business,19,1,9:33:17 AM,19,1,...,10:13:15 AM,13,19.4,NaN,-1.223212,36.893255,-1.275285,36.802702,Rider_Id_447,2107
5,Order_No_13619,User_Id_412,Bike,3,Business,20,3,2:46:37 PM,20,3,...,3:19:29 PM,7,29.6,NaN,-1.303596,36.778378,-1.266837,36.799249,Rider_Id_71,965
6,Order_No_18033,User_Id_1246,Bike,3,Business,14,2,9:59:36 AM,14,2,...,10:34:07 AM,8,15.7,NaN,-1.257147,36.795063,-1.281756,36.763556,Rider_Id_452,1215
7,Order_No_19629,User_Id_3517,Bike,3,Business,2,2,2:07:07 PM,2,2,...,3:27:01 PM,15,29.4,NaN,-1.300921,36.828195,-1.388201,36.769945,Rider_Id_595,1859
8,Order_No_15465,User_Id_267,Bike,3,Personal,11,2,11:33:01 AM,11,2,...,12:18:51 PM,3,24.3,NaN,-1.283311,36.824024,-1.298575,36.808800,Rider_Id_338,2108
9,Order_No_24354,User_Id_1148,Bike,3,Business,7,4,3:34:48 PM,7,4,...,5:03:42 PM,3,29.4,NaN,-1.310059,36.850033,-1.312071,36.856786,Rider_Id_787,860


# functions for pre-processing

In [181]:
def time_from_midnight_in_seconds(dataset, column):
    dataset[column] = pd.to_datetime(dataset[column])
    return (dataset[column] - pd.to_datetime(pd.to_datetime('today').date())).astype('timedelta64[s]')

In [182]:
def delta_time(dataset, higher_time, lower_time):
    return dataset[higher_time] - dataset[lower_time]

In [183]:
def average(list_data):
    return sum(list_data)/len(list_data)+1

In [184]:
def time_to_day_part(time):
    hours = time/3600
    if hours < 6:
        return 'night'
    if hours < 12:
        return 'morning'
    if hours < 18:
        return 'afternoon'
    else:
        return 'evening'

In [185]:
def calculate_bearing(lat1, lng1, lat2, lng2):
    lat1 = np.deg2rad(lat1)
    lat2 = np.deg2rad(lat2)
    diffLong = np.deg2rad(lng2 - lng1)
    x = np.sin(diffLong) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1)
                                       * np.cos(lat2) * np.cos(diffLong))
    initial_bearing = np.arctan2(x, y)
     # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = np.rad2deg(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360
    return compass_bearing

In [186]:
traindata.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,...,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,...,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,...,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,...,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,...,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [187]:
# converting time in seconds from midnight for train data
traindata['Placement - Time'] = time_from_midnight_in_seconds(traindata, 'Placement - Time')
traindata['Confirmation - Time'] = time_from_midnight_in_seconds(traindata, 'Confirmation - Time')
traindata['Pickup - Time'] = time_from_midnight_in_seconds(traindata, 'Pickup - Time')
traindata['Arrival at Pickup - Time'] = time_from_midnight_in_seconds(traindata, 'Arrival at Pickup - Time')
traindata['Arrival at Destination - Time'] = time_from_midnight_in_seconds(traindata, 'Arrival at Destination - Time')

In [188]:
traindata.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,34546.0,9,5,...,38395.0,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,40576.0,12,5,...,44242.0,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,45565.0,30,2,...,46838.0,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,33934.0,15,5,...,36327.0,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,35718.0,13,1,...,37537.0,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [189]:
# converting time in seconds from midnight for test data
testdata['Placement - Time'] = time_from_midnight_in_seconds(testdata, 'Placement - Time')
testdata['Confirmation - Time'] = time_from_midnight_in_seconds(testdata, 'Confirmation - Time')
testdata['Pickup - Time'] = time_from_midnight_in_seconds(testdata, 'Pickup - Time')
testdata['Arrival at Pickup - Time'] = time_from_midnight_in_seconds(testdata, 'Arrival at Pickup - Time')

In [190]:
testdata.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_5073,User_Id_2200,Bike,3,Business,25,4,58736.0,25,4,...,6:22:17 PM,8,NaN,NaN,-1.301446,36.766138,-1.290894,36.822971,Rider_Id_436,1861
1,Order_No_16569,User_Id_972,Bike,3,Business,3,1,44896.0,3,1,...,2:34:37 PM,15,20.7,NaN,-1.286203,36.815297,-1.308100,36.699964,Rider_Id_272,5216
2,Order_No_17965,User_Id_2587,Bike,3,Business,5,5,60648.0,5,5,...,5:59:46 PM,22,26.2,NaN,-1.288780,36.816831,-1.316554,36.700067,Rider_Id_456,1997
3,Order_No_20647,User_Id_2483,Bike,1,Business,16,1,45206.0,16,1,...,1:53:55 PM,12,19.4,NaN,-1.255956,36.789867,-1.284269,36.739776,Rider_Id_536,3949
4,Order_No_23907,User_Id_1502,Bike,3,Business,19,1,34397.0,19,1,...,10:13:15 AM,13,19.4,NaN,-1.223212,36.893255,-1.275285,36.802702,Rider_Id_447,2107


In [191]:
# calculating delta_time for train data
delta_confirm_place_train = delta_time(traindata, 'Confirmation - Time', 'Placement - Time')
delta_pick_arr_confirm_train = delta_time(traindata, 'Arrival at Pickup - Time', 'Confirmation - Time')
delta_pickup_confirm_train = delta_time(traindata, 'Pickup - Time', 'Arrival at Pickup - Time')
delta_arrival_pickup_train = delta_time(traindata, 'Arrival at Destination - Time', 'Pickup - Time')
delta_placement_arrival_train = delta_time(traindata, 'Arrival at Destination - Time', 'Placement - Time')

In [192]:
traindata.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,34546.0,9,5,...,38395.0,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,40576.0,12,5,...,44242.0,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,45565.0,30,2,...,46838.0,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,33934.0,15,5,...,36327.0,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,35718.0,13,1,...,37537.0,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [193]:
# calculating delta_time for test data
delta_confirm_place_test = delta_time(testdata, 'Confirmation - Time', 'Placement - Time')
delta_pick_arr_confirm_test = delta_time(testdata, 'Arrival at Pickup - Time', 'Confirmation - Time')
delta_pickup_confirm_test = delta_time(testdata, 'Pickup - Time', 'Arrival at Pickup - Time')

In [194]:
# merging riders data and train data
ridertrain = traindata.merge(riders, on='Rider Id')
ridertest = testdata.merge(riders, on='Rider Id')

In [195]:
ridertrain.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,34546.0,9,5,...,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745,1637,1309,13.8,549
1,Order_No_27440,User_Id_2642,Bike,3,Personal,18,5,56477.0,18,5,...,-1.326774,36.787807,-1.356237,36.904295,Rider_Id_432,2886,1637,1309,13.8,549
2,Order_No_14170,User_Id_3339,Bike,3,Business,31,5,46301.0,31,5,...,-1.255189,36.782203,-1.273412,36.818206,Rider_Id_432,2615,1637,1309,13.8,549
3,Order_No_21727,User_Id_3523,Bike,3,Personal,2,2,25930.0,2,2,...,-1.290315,36.757377,-1.223520,36.802061,Rider_Id_432,2986,1637,1309,13.8,549
4,Order_No_8301,User_Id_87,Bike,2,Personal,22,2,38458.0,22,2,...,-1.273524,36.799220,-1.300431,36.752427,Rider_Id_432,1602,1637,1309,13.8,549


In [196]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 29 columns):
Order No                                     21201 non-null object
User Id                                      21201 non-null object
Vehicle Type                                 21201 non-null object
Platform Type                                21201 non-null int64
Personal or Business                         21201 non-null object
Placement - Day of Month                     21201 non-null int64
Placement - Weekday (Mo = 1)                 21201 non-null int64
Placement - Time                             21201 non-null float64
Confirmation - Day of Month                  21201 non-null int64
Confirmation - Weekday (Mo = 1)              21201 non-null int64
Confirmation - Time                          21201 non-null float64
Arrival at Pickup - Day of Month             21201 non-null int64
Arrival at Pickup - Weekday (Mo = 1)         21201 non-null int64
Arrival at Pickup - Time 

In [197]:
# Treat Outliers in Training Data
ridertrain = ridertrain[ridertrain['Placement - Weekday (Mo = 1)'] == ridertrain['Confirmation - Weekday (Mo = 1)']]
ridertrain = ridertrain[ridertrain['Placement - Day of Month'] == ridertrain['Confirmation - Day of Month']]

In [198]:
# input missing values for Temperature  for train and test data
ridertrain['Temperature'].fillna(ridertrain['Temperature'].mean(), inplace=True)
ridertest['Temperature'].fillna(ridertest['Temperature'].mean(), inplace=True)

# pre-processing

In [199]:
ridertrain.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,34546.0,9,5,...,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745,1637,1309,13.8,549
1,Order_No_27440,User_Id_2642,Bike,3,Personal,18,5,56477.0,18,5,...,-1.326774,36.787807,-1.356237,36.904295,Rider_Id_432,2886,1637,1309,13.8,549
2,Order_No_14170,User_Id_3339,Bike,3,Business,31,5,46301.0,31,5,...,-1.255189,36.782203,-1.273412,36.818206,Rider_Id_432,2615,1637,1309,13.8,549
3,Order_No_21727,User_Id_3523,Bike,3,Personal,2,2,25930.0,2,2,...,-1.290315,36.757377,-1.223520,36.802061,Rider_Id_432,2986,1637,1309,13.8,549
4,Order_No_8301,User_Id_87,Bike,2,Personal,22,2,38458.0,22,2,...,-1.273524,36.799220,-1.300431,36.752427,Rider_Id_432,1602,1637,1309,13.8,549


In [200]:
# I will later use frequency encoding to handle UserID and Rider ID but let me remove it now

In [201]:
ridertrain.drop(columns=['Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
                         'Arrival at Pickup - Day of Month','Arrival at Pickup - Weekday (Mo = 1)',
                         'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)','Arrival at Destination - Day of Month',
                         'Arrival at Destination - Weekday (Mo = 1)','Vehicle Type', 'Order No','Platform Type', 'User Id', 'Rider Id',
                         'Personal or Business','Arrival at Destination - Time',
                         'Precipitation in millimeters'], inplace=True)

In [202]:
ridertest.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_5073,User_Id_2200,Bike,3,Business,25,4,58736.0,25,4,...,-1.301446,36.766138,-1.290894,36.822971,Rider_Id_436,1861,1202,576,13.4,105
1,Order_No_12353,User_Id_3080,Bike,3,Business,13,3,54043.0,13,3,...,-1.255189,36.782203,-1.266845,36.794064,Rider_Id_436,893,1202,576,13.4,105
2,Order_No_6354,User_Id_633,Bike,3,Business,12,2,55914.0,12,2,...,-1.299144,36.752880,-1.300406,36.829741,Rider_Id_436,389,1202,576,13.4,105
3,Order_No_19613,User_Id_393,Bike,3,Business,22,3,38206.0,22,3,...,-1.322800,36.830644,-1.263818,36.793006,Rider_Id_436,230,1202,576,13.4,105
4,Order_No_13334,User_Id_1882,Bike,3,Business,19,2,57191.0,19,2,...,-1.295688,36.786184,-1.259102,36.800577,Rider_Id_436,4408,1202,576,13.4,105


In [203]:
ridertest.drop(['Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)',
                'Arrival at Pickup - Day of Month', 'Arrival at Pickup - Weekday (Mo = 1)',
                'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)','Vehicle Type','Platform Type','User Id', 'Rider Id',
                'Personal or Business','Arrival at Destination - Time',
                'Precipitation in millimeters'], axis=1, inplace=True)

In [204]:
ridertrain.head()

,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,Pickup - Time,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,9,5,34546.0,34810.0,36287.0,37650.0,4,20.4,-1.317755,36.830370,-1.300406,36.829741,745,1637,1309,13.8,549
1,18,5,56477.0,56490.0,58036.0,58417.0,20,24.5,-1.326774,36.787807,-1.356237,36.904295,2886,1637,1309,13.8,549
2,31,5,46301.0,47569.0,48002.0,48788.0,6,24.7,-1.255189,36.782203,-1.273412,36.818206,2615,1637,1309,13.8,549
3,2,2,25930.0,25949.0,27324.0,27526.0,18,15.2,-1.290315,36.757377,-1.223520,36.802061,2986,1637,1309,13.8,549
4,22,2,38458.0,38544.0,39360.0,39580.0,7,19.2,-1.273524,36.799220,-1.300431,36.752427,1602,1637,1309,13.8,549


In [205]:
ridertrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21199 entries, 0 to 21200
Data columns (total 17 columns):
Placement - Day of Month        21199 non-null int64
Placement - Weekday (Mo = 1)    21199 non-null int64
Placement - Time                21199 non-null float64
Confirmation - Time             21199 non-null float64
Arrival at Pickup - Time        21199 non-null float64
Pickup - Time                   21199 non-null float64
Distance (KM)                   21199 non-null int64
Temperature                     21199 non-null float64
Pickup Lat                      21199 non-null float64
Pickup Long                     21199 non-null float64
Destination Lat                 21199 non-null float64
Destination Long                21199 non-null float64
Time from Pickup to Arrival     21199 non-null int64
No_Of_Orders                    21199 non-null int64
Age                             21199 non-null int64
Average_Rating                  21199 non-null float64
No_of_Ratings          

# Model Training

In [206]:
#Importing Neccessary Libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [207]:
testdata = traindata[5000:].reset_index(inplace = False)
testdata.drop(columns='index', inplace=True)

In [208]:
# Splitting
Y = ridertrain['Time from Pickup to Arrival']
X = ridertrain.drop(columns='Time from Pickup to Arrival')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [209]:
rfr = RandomForestRegressor(n_estimators=180, max_depth=110,max_features=3, min_samples_leaf=3,min_samples_split=8,random_state=0)

In [210]:
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=110,
                      max_features=3, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=8,
                      min_weight_fraction_leaf=0.0, n_estimators=180,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [211]:
#Looking at the importance
features = pd.DataFrame()
features['feature'] = ridertrain.columns
features['importance'] = rfr.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)
features.plot(kind='barh', figsize=(25,25))

In [ ]:
# grid_param = {'n_estimators': [150, 160, 170, 180],
#               'max_depth': [80, 90, 100, 110],
#               'max_features': [2, 3],
#               'min_samples_leaf': [3, 4, 5],
#               'min_samples_split': [8, 10, 12],
#              }grid_param = {'n_estimators': [150, 160, 170, 180],
#               'max_depth': [80, 90, 100, 110],
#               'max_features': [2, 3],
#               'min_samples_leaf': [3, 4, 5],
#               'min_samples_split': [8, 10, 12],
#              }

In [ ]:
# gscv = GridSearchCV(
#     estimator=RandomForestRegressor(),
#     param_grid=grid_param,
#     scoring='neg_mean_absolute_error',
#     cv=4,
#     n_jobs=-1)

In [ ]:
# gscv.fit(X_train, y_train)

In [ ]:
# best_parameters = gscv.best_params_
# best_parameters

In [213]:
y_pred = rfr.predict(X_test)

In [217]:
curv = pd.DataFrame({'actual':y_test,'pr':y_pred})
curv

,actual,pr
12067,1802,2036.184963
7658,621,1965.021673
15522,1771,2013.538181
19473,2025,2420.416886
12713,23,1495.303857
8586,1086,1363.940415
4414,3197,1467.890328
15781,1636,900.355788
7028,744,1261.541896
1620,1854,1691.666028


In [214]:
mse = mean_squared_error(y_test,y_pred)
mse

603472.4310851599

In [215]:
rmse = mse**1/2
rmse

301736.21554257994